# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv("Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [3]:
orders.shape

(397924, 14)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

#### 1. Aggregate the amount_spent for unique customers

In [4]:
# Check count of unique customers

orders['CustomerID'].nunique()

4339

In [5]:
# Group total amount spent by customer

customers_agg = orders.groupby('CustomerID').agg({'amount_spent': sum}).sort_values(by= 'amount_spent', ascending=False).reset_index()
customers_agg

,CustomerID,amount_spent
0,14646,280206.02
1,18102,259657.30
2,17450,194550.79
3,16446,168472.50
4,14911,143825.06
...,...,...
4334,17956,12.75
4335,16454,6.90
4336,14792,6.20
4337,16738,3.75


#### 2. Select customers whose aggregated amount_spent is in a given quantile range

In [6]:
# Quantile ranges

percentile75 = customers_agg['amount_spent'].quantile(0.75)
percentile95 = customers_agg['amount_spent'].quantile(0.95)

print('75th percentile: ', percentile75)
print('95th percentile: ', percentile95)
print('Max: ', customers_agg['amount_spent'].quantile(1))

75th percentile:  1661.64
95th percentile:  5840.181999999982
Max:  280206.02


#### 3. Label selected customers as "VIP" or "Preferred"

In [7]:
# Find VIP and Preferred customers

VIP = customers_agg[customers_agg['amount_spent'] > percentile95]
Preferred = customers_agg[customers_agg['amount_spent'].between(percentile75, percentile95)]
print(VIP.shape)
print(Preferred.shape)

(217, 2)
(868, 2)


In [8]:
# Make labeling function to label customers with their status

def VorP(x):
    if x > percentile95:
        return 'VIP'
    elif (x>=percentile75) and (x<= percentile95):
        return 'Preferred'
    else:
        return 'General'

In [9]:
# Apply function

customers_agg['CustomerStatus'] = list(map(VorP, customers_agg['amount_spent']))
customers_agg['CustomerStatus'].value_counts()

General      3254
Preferred     868
VIP           217
Name: CustomerStatus, dtype: int64

#### There are 868 Preferred customers and 217 VIP customers

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [10]:
# Group amount spent by customerid and country

customers_agg2 = orders.groupby(['CustomerID', 'Country']).agg({'amount_spent': sum}).sort_values(by= 'amount_spent', ascending=False).reset_index()
customers_agg2

,CustomerID,Country,amount_spent
0,14646,Netherlands,280206.02
1,18102,United Kingdom,259657.30
2,17450,United Kingdom,194550.79
3,16446,United Kingdom,168472.50
4,14911,EIRE,143825.06
...,...,...,...
4342,17956,United Kingdom,12.75
4343,16454,United Kingdom,6.90
4344,14792,United Kingdom,6.20
4345,16738,United Kingdom,3.75


In [11]:
# Apply labeling function from before

customers_agg2['CustomerStatus'] = list(map(VorP, customers_agg2['amount_spent']))
customers_agg2

,CustomerID,Country,amount_spent,CustomerStatus
0,14646,Netherlands,280206.02,VIP
1,18102,United Kingdom,259657.30,VIP
2,17450,United Kingdom,194550.79,VIP
3,16446,United Kingdom,168472.50,VIP
4,14911,EIRE,143825.06,VIP
...,...,...,...,...
4342,17956,United Kingdom,12.75,General
4343,16454,United Kingdom,6.90,General
4344,14792,United Kingdom,6.20,General
4345,16738,United Kingdom,3.75,General


In [12]:
# Filter and group vip customers by country

vipcustomers = customers_agg2[customers_agg2['CustomerStatus'] == 'VIP']
vipcustomers.groupby('Country').agg({'CustomerStatus': 'count'}).sort_values(by= 'CustomerStatus', ascending=False)

,CustomerStatus
Country,
United Kingdom,177
Germany,10
France,9
Switzerland,3
Spain,2
Portugal,2
Japan,2
EIRE,2
Finland,1


#### United Kingdom has the most VIP customers with 177

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [13]:
# Filter vip and preferred customers

vip_preferred_customers = customers_agg2[(customers_agg2['CustomerStatus'] == 'VIP')|(customers_agg2['CustomerStatus'] == 'Preferred')]
vip_preferred_customers 

,CustomerID,Country,amount_spent,CustomerStatus
0,14646,Netherlands,280206.02,VIP
1,18102,United Kingdom,259657.30,VIP
2,17450,United Kingdom,194550.79,VIP
3,16446,United Kingdom,168472.50,VIP
4,14911,EIRE,143825.06,VIP
...,...,...,...,...
1080,17656,United Kingdom,1674.69,Preferred
1081,16115,United Kingdom,1667.97,Preferred
1082,12912,United Kingdom,1662.30,Preferred
1083,12530,Germany,1662.28,Preferred


In [14]:
# Group vip and preferred customers by country

vip_preferred_customers.groupby('Country').agg({'CustomerStatus': 'count'}).sort_values(by= 'CustomerStatus', ascending=False).head()

,CustomerStatus
Country,
United Kingdom,932
Germany,39
France,29
Belgium,11
Switzerland,9


#### United Kingdom also has the most VIP and Preferred customers combined with 932 

In [15]:
# We can also test this with country value counts from our filtered list

vip_preferred_customers['Country'].value_counts()

United Kingdom     932
Germany             39
France              29
Belgium             11
Switzerland          9
Portugal             7
Norway               7
Spain                7
Finland              5
Italy                5
Japan                4
Channel Islands      4
Australia            4
Denmark              3
Cyprus               3
EIRE                 3
Sweden               2
Israel               2
Malta                1
Canada               1
Greece               1
Austria              1
Netherlands          1
Poland               1
Iceland              1
Singapore            1
Lebanon              1
Name: Country, dtype: int64